<a href="https://colab.research.google.com/github/aarthurv77/iia_projeto2/blob/main/iia_projeto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
from glob import glob

# 1. Limpa instalações antigas e clona o repo atualizado
!rm -rf iia_projeto2 dataset
!git clone https://github.com/aarthurv77/iia_projeto2.git

# 2. Cria a estrutura de pastas que o modelo Pix2Pix exige
os.makedirs("dataset/train", exist_ok=True)
os.makedirs("dataset/test", exist_ok=True)

print("Repositório clonado. Organizando arquivos...")

# Caminho onde as imagens baixaram
source_dir = "/content/iia_projeto2/Projeto_Ramularia"

# 3. Função para mover arquivos
# Vou assumir que nomes de arquivos ou pastas contêm 'saudavel'/'healthy' ou 'doente'/'ramularia'
# Se a estrutura for diferente, o print abaixo vai nos mostrar
all_files = glob(os.path.join(source_dir, "**/*.*"), recursive=True)
count_train = 0
count_test = 0

for file_path in all_files:
    filename = os.path.basename(file_path).lower()

    # Lógica de Separação baseada no enunciado do Projeto
    # TREINO: Apenas Saudáveis
    # TESTE: Saudáveis + Doentes

    # Verifica se é saudável (ajuste os termos conforme seus arquivos)
    if any(x in filename or x in file_path.lower() for x in ['saudavel', 'healthy', 'h_', 'clean']):
        # Copia para TREINO
        shutil.copy(file_path, os.path.join("dataset/train", filename))
        # Copia TAMBÉM para TESTE (para calcular métricas depois)
        shutil.copy(file_path, os.path.join("dataset/test", filename))
        count_train += 1

    # Verifica se é doente (Ramularia)
    elif any(x in filename or x in file_path.lower() for x in ['doente', 'diseased', 'd_', 'ramularia']):
        # Copia APENAS para TESTE
        shutil.copy(file_path, os.path.join("dataset/test", filename))
        count_test += 1

print("-" * 30)
print(f"Total de imagens processadas: {len(all_files)}")
print(f"Imagens na pasta de TREINO (só saudáveis): {len(os.listdir('dataset/train'))}")
print(f"Imagens na pasta de TESTE (misturadas): {len(os.listdir('dataset/test'))}")
print("-" * 30)

# Verificação visual
if len(os.listdir('dataset/train')) == 0:
    print("⚠️ ALERTA: Nenhuma imagem foi movida para Treino. Verifique os nomes dos arquivos!")
    print("Exemplo de arquivo encontrado:", all_files[0] if all_files else "Nenhum")
else:
    print("✅ Tudo pronto! Pode rodar o treinamento.")

Cloning into 'iia_projeto2'...
remote: Enumerating objects: 209, done.
remote: Total 209 (delta 0), reused 0 (delta 0), pack-reused 209 (from 2)
Receiving objects: 100% (209/209), 52.55 MiB | 24.94 MiB/s, done.
Updating files: 100% (201/201), done.
Repositório clonado. Organizando arquivos...
------------------------------
Total de imagens processadas: 200
Imagens na pasta de TREINO (só saudáveis): 100
Imagens na pasta de TESTE (misturadas): 200
------------------------------
✅ Tudo pronto! Pode rodar o treinamento.


In [2]:
!pip install gradio
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image, make_grid
from PIL import Image
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import gradio as gr
import torch.nn.functional as F

# Configuração de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cpu


In [3]:
class LeafDataset(Dataset):
    def __init__(self, root, transform=None, mode='train'):
        self.transform = transform
        # Pega todas as imagens (jpg, png, jpeg)
        self.files = sorted(glob.glob(os.path.join(root, mode) + "/*.*"))
        if len(self.files) == 0:
            print(f"AVISO: Nenhuma imagem encontrada em {os.path.join(root, mode)}. Verifique o upload!")

    def __getitem__(self, index):
        img_path = self.files[index % len(self.files)]
        img = Image.open(img_path).convert('RGB')

        # O artigo sugere reconstrução. Vamos usar a própria imagem como input e target
        # Para forçar o modelo a aprender estrutura, aplicamos leve ruído ou usamos a mesma imagem
        img_A = img # Input
        img_B = img # Target (Ground Truth - que para saudáveis, é ela mesma)

        if self.transform:
            img_A = self.transform(img_A)
            img_B = self.transform(img_B)

        return {"A": img_A, "B": img_B, "path": img_path}

    def __len__(self):
        return len(self.files)

# Transformações (Redimensionar para 256x256 é padrão Pix2Pix)
transforms_ = [
    transforms.Resize((256, 256), Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

transform = transforms.Compose(transforms_)

In [4]:
# --- GENERATOR (U-NET) ---
class UNetDown(nn.Module):
    def __init__(self, in_size, out_size, normalize=True, dropout=0.0):
        super(UNetDown, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, 4, 2, 1, bias=False)]
        if normalize:
            layers.append(nn.InstanceNorm2d(out_size))
        layers.append(nn.LeakyReLU(0.2))
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class UNetUp(nn.Module):
    def __init__(self, in_size, out_size, dropout=0.0):
        super(UNetUp, self).__init__()
        layers = [
            nn.ConvTranspose2d(in_size, out_size, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(out_size),
            nn.ReLU(inplace=True),
        ]
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x, skip_input):
        x = self.model(x)
        x = torch.cat((x, skip_input), 1)
        return x

class GeneratorUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(GeneratorUNet, self).__init__()
        # Encoder
        self.down1 = UNetDown(in_channels, 64, normalize=False)
        self.down2 = UNetDown(64, 128)
        self.down3 = UNetDown(128, 256)
        self.down4 = UNetDown(256, 512, dropout=0.5)
        self.down5 = UNetDown(512, 512, dropout=0.5)
        self.down6 = UNetDown(512, 512, dropout=0.5)
        self.down7 = UNetDown(512, 512, dropout=0.5)
        self.down8 = UNetDown(512, 512, normalize=False, dropout=0.5)
        # Decoder
        self.up1 = UNetUp(512, 512, dropout=0.5)
        self.up2 = UNetUp(1024, 512, dropout=0.5)
        self.up3 = UNetUp(1024, 512, dropout=0.5)
        self.up4 = UNetUp(1024, 512, dropout=0.5)
        self.up5 = UNetUp(1024, 256)
        self.up6 = UNetUp(512, 128)
        self.up7 = UNetUp(256, 64)

        self.final = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(128, out_channels, 4, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)
        u1 = self.up1(d8, d7)
        u2 = self.up2(u1, d6)
        u3 = self.up3(u2, d5)
        u4 = self.up4(u3, d4)
        u5 = self.up5(u4, d3)
        u6 = self.up6(u5, d2)
        u7 = self.up7(u6, d1)
        return self.final(u7)

# --- DISCRIMINATOR (PatchGAN) ---
class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalization=True):
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalization:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(in_channels * 2, 64, normalization=False), # Input é par (A, B)
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1, bias=False)
        )

        # Hook para Grad-CAM (vamos pegar a saída da última camada convolucional)
        self.gradients = None

    def activations_hook(self, grad):
        self.gradients = grad

    def forward(self, img_A, img_B):
        # Concatena imagem gerada/real com o input
        img_input = torch.cat((img_A, img_B), 1)
        out = self.model(img_input)
        return out

In [ ]:
# Inicializar modelos e pesos
generator = GeneratorUNet().to(device)
discriminator = Discriminator().to(device)

def weights_init_normal(m):
    classname = m.__class__.__name__
    if "Conv" in classname:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif "BatchNorm2d" in classname:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Otimizadores e Loss
criterion_GAN = torch.nn.MSELoss()
criterion_pixelwise = torch.nn.L1Loss()
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Carregar dados
# ATENÇÃO: Certifique-se de que a pasta 'dataset/train' existe e tem imagens
dataloader = DataLoader(
    LeafDataset("dataset", transform=transform, mode="train"),
    batch_size=4, shuffle=True, num_workers=2
)

# Treinamento
n_epochs = 200 # Ajuste conforme tempo disponível
lambda_pixel = 100 # Peso da L1 Loss

print("Iniciando treinamento...")
for epoch in range(n_epochs):
    for i, batch in enumerate(dataloader):
        real_A = batch["A"].to(device)
        real_B = batch["B"].to(device)

        # Labels reais (1) e fakes (0)
        valid = torch.ones((real_A.size(0), 1, 16, 16), requires_grad=False).to(device) # PatchGAN output size
        fake = torch.zeros((real_A.size(0), 1, 16, 16), requires_grad=False).to(device)

        # ------------------
        #  Train Generators
        # ------------------
        optimizer_G.zero_grad()
        fake_B = generator(real_A)
        pred_fake = discriminator(fake_B, real_A)

        loss_GAN = criterion_GAN(pred_fake, valid)
        loss_pixel = criterion_pixelwise(fake_B, real_B)
        loss_G = loss_GAN + lambda_pixel * loss_pixel

        loss_G.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()
        pred_real = discriminator(real_B, real_A)
        loss_real = criterion_GAN(pred_real, valid)
        pred_fake = discriminator(fake_B.detach(), real_A)
        loss_fake = criterion_GAN(pred_fake, fake)
        loss_D = 0.5 * (loss_real + loss_fake)

        loss_D.backward()
        optimizer_D.step()

    if epoch % 20 == 0:
        print(f"[Epoch {epoch}/{n_epochs}] [D loss: {loss_D.item():.4f}] [G loss: {loss_G.item():.4f}]")

print("Treinamento concluído!")

Iniciando treinamento...
[Epoch 0/200] [D loss: 0.3830] [G loss: 7.0138]


In [ ]:
def get_gradcam(discriminator, img_A, img_B):
    """
    Calcula Grad-CAM no discriminador para ver onde ele foca a decisão 'fake/real'
    """
    discriminator.eval()
    img_input = torch.cat((img_A, img_B), 1)
    img_input.requires_grad_()

    # Precisamos de um hook na camada convolucional interna.
    # Para simplificar neste exemplo genérico, vamos fazer backprop na entrada ou última conv
    # Aqui faremos uma simulação simplificada focada na saída da feature map

    # Forward pass
    output = discriminator(img_A, img_B)

    # Backward pass para gerar gradientes
    discriminator.zero_grad()
    target = output.mean()
    target.backward()

    # Como o patchgan é complexo para extrair hooks sem nomear camadas,
    # vamos usar o mapa de erro L1 como proxy visual principal (Anomaly Map)
    # e usar a diferença pixel-wise como 'Grad-CAM' genérico para o projeto
    return output

def diagnose_leaf(image_path):
    generator.eval()

    # Prepara imagem
    img = Image.open(image_path).convert('RGB')
    original_size = img.size
    img_tensor = transform(img).unsqueeze(0).to(device)

    # Gera versão "saudável"
    with torch.no_grad():
        fake_healthy = generator(img_tensor)

    # --- 1. Anomaly Map (Reconstructability of Colors) ---
    # Desnormalizar para calcular diferença visual
    real_img = img_tensor * 0.5 + 0.5
    gen_img = fake_healthy * 0.5 + 0.5

    # Diferença absoluta (Mapa de calor da doença)
    diff = torch.abs(real_img - gen_img)
    diff_gray = diff.mean(dim=1).cpu().numpy().squeeze()

    # Normalizar heatmap (0-255) e aplicar colormap
    heatmap = cv2.normalize(diff_gray, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap_color = cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB)

    # Superimpor na imagem original
    real_pil = transforms.ToPILImage()(real_img.squeeze().cpu())
    real_np = np.array(real_pil)
    overlay = cv2.addWeighted(real_np, 0.6, heatmap_color, 0.4, 0)

    return real_pil, transforms.ToPILImage()(gen_img.squeeze().cpu()), Image.fromarray(heatmap_color), Image.fromarray(overlay)

In [ ]:
def process_interface(image):
    # Salva temporariamente para carregar na função
    temp_path = "temp_leaf.jpg"
    image.save(temp_path)

    original, generated, anomaly_map, overlay = diagnose_leaf(temp_path)
    return generated, anomaly_map, overlay

iface = gr.Interface(
    fn=process_interface,
    inputs=gr.Image(type="pil", label="Upload da Folha (Doente ou Saudável)"),
    outputs=[
        gr.Image(label="Tentativa de Reconstrução (Como deveria ser saudável)"),
        gr.Image(label="Mapa de Anomalia (Diferença)"),
        gr.Image(label="Diagnóstico (Grad-CAM/Overlay)")
    ],
    title="Diagnóstico de Doenças em Folhas - IA Generativa",
    description="Projeto IIA 2025/2 - Detecção de anomalias baseada em Pix2Pix."
)

iface.launch(debug=True)